In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('IFR_Extract_with_selected_columns_15-5-23.csv')

obreak_date = pd.to_datetime(data.obreak_date)
datebone = pd.to_datetime(data.datebone)
y = ( abs( datebone - obreak_date))
X = data.drop(["obreak_date","datebone"],axis=1)




In [15]:
cat_features = ['parentbreak', 'alcohol',
                'arthritis', 'diabetes',
                'oralster', 'smoke', 'obreak',
                # These features were determined to apply minimal impact even
                # 'respdisease', 'hbp','heartdisease',
                # 'ptunsteady', 'wasfractdue2fall', 'cholesterol',
                # 'ptfall', 'shoulder', 'wrist', 'bmdtest_10yr_caroc'
                ]
for feature in cat_features:
    if data is not None:
        cat_one_hot = pd.get_dummies(data[feature], prefix=f'{feature}', drop_first=False)
        data = data.drop(feature, axis=1)
        data = data.join(cat_one_hot)



None

In [11]:
X = encode_cat_data(data)
X

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
selectedColumns = ['PatientAge', "PatientGender", 'bmdtest_weight', 'bmdtest_height']


In [2]:
dropList = []
for i in data:
    if data[i].dtypes == 'O':
        dropList.append(data[i].name)
dropList.append("CompletedSurveyId")
dropList.append("PatientId")
X = data.drop(dropList,axis=1)
X.fillna(0,inplace=True)
y = pd.DataFrame({"time":y})

In [15]:
y.time

y['event'] = y.time.apply(lambda x: x.days != 0 )
structured_array = y.to_records(index=False)

swapped = pd.DataFrame({
    "event": y.event,
    "time": y.time.apply(lambda x: x.days)
})
(swapped.event).value_counts()

True     792
False      8
Name: event, dtype: int64

In [4]:
swapped.event = swapped.event.astype(bool)
swapped.event
structured_array = np.rec.array(swapped.to_records(index=False))

In [9]:
mergedData = pd.concat([X,swapped],axis=1)
mergedData

,PatientGender,PatientAge,screeningmode,disclaimer_new,howbreak,eligible_hidden,consent_new,able2complete,able2complete_no,translator,...,FraxRiskPercentage,FraxRiskLevel,CarocWithCF,CarocWithSQ,Prnt_requis_sent_bmdr,Prnt_test_done_bmdr,Prnt_results_rcvd_bmdr,Chld_task_secSpecialist_referral,event,time
0,1,53,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,524
1,1,85,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,2046
2,1,90,1,1,1,1,1,0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,15455
3,1,81,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,4354
4,1,60,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,2207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1,83,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,579
796,1,60,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,5109
797,2,76,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,2125
798,1,61,1,1,1,1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,518


In [14]:
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.feature_selection import SelectKBest, f_regression

# Perform univariate analysis
univariate_selector = SelectKBest(score_func=f_regression, k=10)
univariate_selector.fit(X, swapped['time'])
selected_features = X.columns[univariate_selector.get_support()]

# Perform multivariate analysis using Cox proportional hazards regression
cph = CoxPHFitter()
formula = 'SurvivalTime ~ ' + ' + '.join(selected_features)
cph.fit(mergedData, duration_col='time', event_col='event', formula='time ~ ' + ' + '.join(selected_features))

# Get the coefficient values and p-values of the selected features
feature_summary = cph.summary
selected_features_summary = feature_summary.loc[selected_features, ['coef', 'p']]

FactorEvaluationError: Unable to evaluate factor `time`. [KeyError: 'time']